# DDSP for Guitar: Google Colab チュートリアル

DDSP for Guitar チュートリアルへようこそ！このノートブックでは、環境のセットアップ、ダミーデータセットの作成、モデルのトレーニング、そして音声の合成までの一連の流れをご案内します。

## 概要

このプロジェクトは、エレキギターの音声を合成するための微分可能デジタル信号処理（DDSP）の実装です。以下の機能を備えています:

- ハーモニック合成（倍音生成）
- ノイズ合成
- ウェーブシェイパー（非線形歪み）
- トーンスタック（3バンドEQ）
- トランジェント分離

## 必要な環境

- Google Colab (GPU推奨)
- Python 3.8以上

## 1. 環境のセットアップ

まず、リポジトリをクローンして必要な依存関係をインストールします。

**重要:** 次のセルで `<YOUR_REPOSITORY_URL>` を実際のリポジトリのURLに置き換えてください（例: `https://github.com/username/ddsp_guitar.git`）。

In [ ]:
# リポジトリをクローン
!git clone <YOUR_REPOSITORY_URL>
%cd ddsp_guitar

# 依存関係をインストール
!pip install -r requirements.txt

# GPU使用可否を確認
import torch
print(f"PyTorch バージョン: {torch.__version__}")
print(f"CUDA 利用可能: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU デバイス: {torch.cuda.get_device_name(0)}")

## 2. ダミーデータセットの作成

トレーニングに使用するダミーデータセットを作成します。実際の使用では、エレキギターのDI（Direct Input）音源を用意してください。

ここでは、サイン波をベースにした合成音声を生成してデータセットとして使用します。

In [ ]:
# ダミーデータセットを作成
!python scripts/create_dummy_dataset.py \
    --output_dir dataset/train/audio \
    --num_samples 50 \
    --sample_rate 48000 \
    --duration 2.0

# 検証用データも作成
!python scripts/create_dummy_dataset.py \
    --output_dir dataset/val/audio \
    --num_samples 10 \
    --sample_rate 48000 \
    --duration 2.0

print("\nデータセットの作成が完了しました！")
print("トレーニングデータ: dataset/train/audio/")
print("検証データ: dataset/val/audio/")

## 3. モデルのトレーニング

データセットが準備できたので、モデルをトレーニングします。

**注意:** 完全なトレーニングには時間がかかるため、ここでは少ないエポック数で実行します。実際のプロジェクトでは、より多くのエポック（100エポック以上）を推奨します。

In [ ]:
# トレーニングを実行（デモ用に10エポック）
!python scripts/train.py \
    --train_dir dataset/train/audio \
    --val_dir dataset/val/audio \
    --sample_rate 48000 \
    --segment_seconds 1.0 \
    --batch_size 4 \
    --epochs 10 \
    --lr 1e-4 \
    --num_harm 64 \
    --num_noise 8 \
    --checkpoint_dir checkpoints \
    --log_dir logs \
    --save_every 5 \
    --device cuda

print("\nトレーニングが完了しました！")
print("チェックポイント: checkpoints/")
print("TensorBoardログ: logs/")

## 4. トレーニング結果の確認

TensorBoardを使用してトレーニングの進捗を確認します。


In [ ]:
# TensorBoardを起動
%load_ext tensorboard
%tensorboard --logdir logs

# 最新のログディレクトリを表示
import os
from pathlib import Path

log_dir = Path('logs')
if log_dir.exists():
    log_dirs = sorted(log_dir.glob('*'))
    if log_dirs:
        print(f"最新のログ: {log_dirs[-1]}")


## 5. モデルの推論

トレーニングしたモデルを使って音声を合成してみましょう。


In [ ]:
import torch
from ddsp_guitar.model import GuitarDDSP
import soundfile as sf
import numpy as np
from IPython.display import Audio
from pathlib import Path

# 設定
SAMPLE_RATE = 48000
AUDIO_LENGTH_SECONDS = 2
FRAME_RATE = 250
HOP_SIZE = SAMPLE_RATE // FRAME_RATE

# デバイスの設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用デバイス: {device}")

# モデルの読み込み
model = GuitarDDSP(sample_rate=SAMPLE_RATE, num_harm=64, num_noise=8, device=device)

# チェックポイントから重みを読み込み（ベストモデルまたは最終モデル）
checkpoint_path = Path('checkpoints/best_model.pt')
if not checkpoint_path.exists():
    checkpoint_path = Path('checkpoints/final_model.pt')

if checkpoint_path.exists():
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"チェックポイントを読み込みました: {checkpoint_path}")
else:
    print("チェックポイントが見つかりません。初期化されたモデルを使用します。")

model.eval()
print(f"モデルパラメータ数: {sum(p.numel() for p in model.parameters()):,}")


### 5.1 テスト音声の合成

ダミーの入力パラメータを使って音声を合成します。実際の使用では、入力音声から特徴量（基本周波数、ラウドネスなど）を抽出します。


In [ ]:
# 入力パラメータの作成
batch_size = 1
audio_length_samples = AUDIO_LENGTH_SECONDS * SAMPLE_RATE
n_frames = audio_length_samples // HOP_SIZE

# 条件付け入力（通常は入力音声から抽出される特徴量）
conditioning_input = torch.randn(batch_size, n_frames).to(device)

# 基本周波数（f0）- A2音（110Hz）からE3音（165Hz）へのスライド
f0_start = 110.0  # A2
f0_end = 165.0    # E3
f0_values = torch.linspace(f0_start, f0_end, n_frames).unsqueeze(0).to(device)

# ラウドネス - 時間とともに減衰
loudness_values = torch.linspace(-20.0, -40.0, n_frames).unsqueeze(0).to(device)

print(f"入力形状:")
print(f"  conditioning_input: {conditioning_input.shape}")
print(f"  f0: {f0_values.shape}")
print(f"  loudness: {loudness_values.shape}")

# 音声を合成
print("\n音声を合成中...")
with torch.no_grad():
    output_audio = model(conditioning_input, f0_values, loudness_values)

print("合成完了！")
print(f"出力音声形状: {output_audio.shape}")

# CPUに移動して保存
output_audio_np = output_audio.cpu().numpy().squeeze()

# 正規化
output_audio_np = output_audio_np / (np.abs(output_audio_np).max() + 1e-8) * 0.9

# 保存
output_filename = 'synthesized_guitar.wav'
sf.write(output_filename, output_audio_np, SAMPLE_RATE)
print(f"音声を保存しました: {output_filename}")

# 再生
Audio(output_audio_np, rate=SAMPLE_RATE)


### 5.2 メロディの合成

複数の音符を使ってシンプルなメロディを合成してみましょう。


In [ ]:
# メロディの定義（音符と長さ）
# C4, D4, E4, F4, G4, A4, B4, C5（ドレミファソラシド）
melody_notes = [
    (261.63, 0.3),  # C4
    (293.66, 0.3),  # D4
    (329.63, 0.3),  # E4
    (349.23, 0.3),  # F4
    (392.00, 0.3),  # G4
    (440.00, 0.3),  # A4
    (493.88, 0.3),  # B4
    (523.25, 0.5),  # C5（最後の音は長く）
]

# メロディ全体の長さを計算
total_duration = sum(duration for _, duration in melody_notes)
total_samples = int(total_duration * SAMPLE_RATE)
total_frames = total_samples // HOP_SIZE

# f0とラウドネスのシーケンスを作成
f0_sequence = []
loudness_sequence = []

current_frame = 0
for freq, duration in melody_notes:
    num_frames = int((duration * SAMPLE_RATE) // HOP_SIZE)
    
    # 各音符のf0（一定）
    f0_sequence.extend([freq] * num_frames)
    
    # ラウドネス（アタック→サスティン→リリース）
    attack_frames = num_frames // 4
    sustain_frames = num_frames // 2
    release_frames = num_frames - attack_frames - sustain_frames
    
    attack = np.linspace(-35, -15, attack_frames)
    sustain = np.full(sustain_frames, -15)
    release = np.linspace(-15, -45, release_frames)
    
    loudness_sequence.extend(attack.tolist())
    loudness_sequence.extend(sustain.tolist())
    loudness_sequence.extend(release.tolist())

# テンソルに変換
f0_tensor = torch.tensor(f0_sequence, dtype=torch.float32).unsqueeze(0).to(device)
loudness_tensor = torch.tensor(loudness_sequence, dtype=torch.float32).unsqueeze(0).to(device)
conditioning_tensor = torch.randn(1, len(f0_sequence)).to(device)

print(f"メロディの長さ: {total_duration:.2f}秒")
print(f"フレーム数: {len(f0_sequence)}")

# メロディを合成
print("\nメロディを合成中...")
with torch.no_grad():
    melody_audio = model(conditioning_tensor, f0_tensor, loudness_tensor)

melody_audio_np = melody_audio.cpu().numpy().squeeze()
melody_audio_np = melody_audio_np / (np.abs(melody_audio_np).max() + 1e-8) * 0.9

# 保存
melody_filename = 'synthesized_melody.wav'
sf.write(melody_filename, melody_audio_np, SAMPLE_RATE)
print(f"メロディを保存しました: {melody_filename}")

# 再生
Audio(melody_audio_np, rate=SAMPLE_RATE)


## 6. 次のステップ

このチュートリアルでは基本的な使い方を紹介しましたが、より高度な使い方には以下があります：

### 6.1 実際のギター音源を使う

ダミーデータの代わりに、実際のエレキギターのDI（Direct Input）音源を使用してトレーニングすることで、より高品質な結果が得られます。

```python
# 実際のデータセットの準備
# 1. wavファイルをdataset/train/audio/に配置
# 2. 同様にdataset/val/audio/に検証データを配置
# 3. トレーニングスクリプトを実行
```

### 6.2 ハイパーパラメータの調整

以下のパラメータを調整することで、モデルの性能を向上させることができます：

- `num_harm`: ハーモニクス数（デフォルト: 64）
- `num_noise`: ノイズバンド数（デフォルト: 8）
- `lr`: 学習率（デフォルト: 1e-4）
- `batch_size`: バッチサイズ（GPUメモリに応じて調整）
- `epochs`: エポック数（100エポック以上推奨）

### 6.3 リアルタイム処理

`ddsp_guitar/rt/engine.py`を使用することで、リアルタイムでの音声処理が可能です。

### 6.4 モデルの保存とエクスポート

```python
# モデルの保存
torch.save(model.state_dict(), 'guitar_ddsp_model.pth')

# モデルの読み込み
model.load_state_dict(torch.load('guitar_ddsp_model.pth'))
```

## 参考資料

- [DDSP論文](https://arxiv.org/abs/2001.04643)
- [プロジェクトリポジトリ](https://github.com/your-repo/ddsp_guitar)
- [ドキュメント](docs/)


## 7. トラブルシューティング

### GPU メモリ不足の場合

バッチサイズを減らすか、セグメント長を短くしてください：

```bash
!python scripts/train.py --batch_size 2 --segment_seconds 0.5
```

### トレーニングが不安定な場合

学習率を下げるか、勾配クリッピングを調整してください：

```bash
!python scripts/train.py --lr 5e-5 --grad_clip 0.5
```

### 音質が悪い場合

- より多くのデータでトレーニングする
- エポック数を増やす（100エポック以上）
- ハーモニクス数を増やす（`--num_harm 128`）
- 実際のギター音源を使用する

## おわりに

このチュートリアルを通じて、DDSP for Guitarの基本的な使い方を学びました。
質問や問題がある場合は、GitHubのIssueで報告してください。

Happy coding!
